In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [3]:
from collections import defaultdict, Counter
from sklearn.model_selection import KFold
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from nltk.corpus import stopwords

import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

In [8]:
import numpy as np
import pandas as pd

data_t = pd.read_csv("mtsamples_t2.csv")

temp = [i for i in data_t.index if pd.isna(data_t["transcription"][i])]
data_t = data_t.drop(temp)
data_t = data_t.reset_index(drop=True)
print(data_t["medical_specialty"].unique())

d = {'group_1': 1, 'group_2': 2, 'group_3': 3}
label = data_t['medical_specialty'].map(d, na_action='ignore')
print(label)

['group_3' 'group_2' 'group_1']
0       3
1       2
2       2
3       1
4       1
       ..
4961    3
4962    3
4963    3
4964    3
4965    3
Name: medical_specialty, Length: 4966, dtype: int64


In [9]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)


# Loading tokenizer from the bert layer
PRE_TRAINED_MODEL_NAME = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

INFO:absl:Using C:\Users\Admin\AppData\Local\Temp\tfhub_modules to cache modules.


In [10]:
import string  
punct = set(string.punctuation)
nlp = spacy.load("en_core_sci_sm")
med7 = spacy.load("en_core_med7_lg")

In [11]:
def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        temp.append(token)
#     temp = getNGrams(temp,5)
    temp = " ".join(temp)
    return temp

In [12]:
#thai
from random import sample, randrange 
import copy
g1 = list(data_t[data_t["medical_specialty"] == "group_1"].index.values)
g2 = list(data_t[data_t["medical_specialty"] == "group_2"].index.values)
g3 = list(data_t[data_t["medical_specialty"] == "group_3"].index.values)
g1_temp = copy.deepcopy(g1)
g2_temp = copy.deepcopy(g2)
g3_temp = copy.deepcopy(g3)

t1 = [g1_temp.pop(randrange(len(g1_temp))) for _ in range(1220)]
t2 = [g2_temp.pop(randrange(len(g2_temp))) for _ in range(1220)]
t3 = [g3_temp.pop(randrange(len(g3_temp))) for _ in range(1220)]

x_train = []
y_train = []
x_test = []
y_test = []
train_index = t1+t2+t3
test_index = g1_temp + g2_temp + g3_temp

X_train, X_test = data_t['transcription'][train_index], data_t['transcription'][test_index]
y_train, y_test = data_t['medical_specialty'][train_index], data_t['medical_specialty'][test_index]

In [13]:
X_train = [tokenize_doc(x) for x in X_train]
X_test = [tokenize_doc(x) for x in X_test]
# data['transcription'].apply((lambda x: tokenize_doc(x) ))

In [14]:
d = {'group_1': 1, 'group_2': 2, 'group_3': 3}
y_train = y_train.map(d, na_action='ignore')
y_test = y_test.map(d, na_action='ignore')

In [15]:
print(X_train[0])

discharge diagnoses:,1 acute cerebrovascular accident/left basal ganglia deep white matter left parietal lobe.,2 hypertension.,3 urinary tract infection.,4 portable chest impression atherosclerotic change aortic knob.,2 chest portable impression mild tortuosity thoracic aorta maybe hypertension right lateral costophrenic angle swallowing study normal swallowing study minimal penetration thin liquids.,4 head ct air-fluid level right maxillary sinus acute sinusitis 1.8-cm oval low density mass dependent portion left maxillary sinus retention mucoparietal cell right maxillary sinus ethmoid sinuses.,4 iv ct scan head neck brain changes consistent with infarct right basal ganglia deep white matter left parietal lobe diffuse smooth narrowing left middle cerebral artery congenital abnormality clinical correlation necessary.,6 echocardiogram bubble study impression normal left ventricular systolic function left ventricular ejection fraction mild concentric left ventricular hypertrophy left atr

In [16]:
# tokenized = pd.Series(tokenized)
# tokenized = tokenized.apply((lambda x: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))))
tokenized_train = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))[0:510] for x in X_train]
tokenized_test = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))[0:510] for x in X_test]
# # Reshaping the array:
# max_len = 0
# for i in tokenized:
#     if len(i) > max_len:
#         max_len = len(i)
# print(max_len)

In [17]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [69]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(tokeniz_s, label, test_size=0.33, random_state=42)

In [18]:
print(X_train[0])

discharge diagnoses:,1 acute cerebrovascular accident/left basal ganglia deep white matter left parietal lobe.,2 hypertension.,3 urinary tract infection.,4 portable chest impression atherosclerotic change aortic knob.,2 chest portable impression mild tortuosity thoracic aorta maybe hypertension right lateral costophrenic angle swallowing study normal swallowing study minimal penetration thin liquids.,4 head ct air-fluid level right maxillary sinus acute sinusitis 1.8-cm oval low density mass dependent portion left maxillary sinus retention mucoparietal cell right maxillary sinus ethmoid sinuses.,4 iv ct scan head neck brain changes consistent with infarct right basal ganglia deep white matter left parietal lobe diffuse smooth narrowing left middle cerebral artery congenital abnormality clinical correlation necessary.,6 echocardiogram bubble study impression normal left ventricular systolic function left ventricular ejection fraction mild concentric left ventricular hypertrophy left atr

In [19]:
MAX_LEN = 510

In [20]:
train_input = bert_encode(X_train, tokenizer, max_len= MAX_LEN)
# encode  test set 
test_input = bert_encode(X_test, tokenizer, max_len= MAX_LEN )
train_labels = y_train

In [21]:
train_labels = y_train.to_numpy()

In [22]:
from tensorflow.keras.layers import  Input
input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

In [23]:
from tensorflow.keras.layers import Dense
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])  
clf_output = sequence_output[:, 0, :]
out = Dense(1, activation='sigmoid')(clf_output)  

In [24]:
# intilize model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 510)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 510)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 510)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [25]:
print(train_labels)

[1 1 1 ... 3 3 3]


In [26]:
# train
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=2
)


Epoch 1/2
92/92 [==============================] - 3942s 43s/step - loss: 0.0317 - accuracy: 0.4167 - val_loss: -0.7555 - val_accuracy: 0.0000e+00
Epoch 2/2
92/92 [==============================] - 3896s 42s/step - loss: -0.0013 - accuracy: 0.4167 - val_loss: -0.8260 - val_accuracy: 0.0000e+00


In [27]:
test_pred = model.predict(test_input)
preds = test_pred.round().astype(int)
# print(metrics.classification_report(preds, y_test.to_numpy(), digits=5))

NameError: name 'metrics' is not defined

In [ ]:
from sklearn.metrics import classification_report
print(metrics.classification_report(preds, y_test.to_numpy(), digits=5))